In [1]:
# nvidia-smi
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:

import numpy as np
import torch
import torch.optim as optim
import torch.nn.functional as F
import multiprocessing
import time

from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from torch.nn import Sequential as Seq, Linear, Parameter, ReLU
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from utils.preprocessing_for_mesh import create_mesh

In [3]:
# load image file paths
svg_folder = './datasets/svg'
png_folder = './datasets/png'
imgs = []
png = []
dataset = []

for root, folders, files in os.walk(svg_folder):
    for file in files:
        if file.split('.')[1] != 'svg': continue
        if 'checkpoint' in file: continue
        
        file_path = os.path.join(svg_folder, file)
        imgs.append(file_path)
        
        file_path = os.path.join(png_folder, file.replace('svg', 'png'))
        png.append(file_path)

In [4]:
# for i, file_path in enumerate(tqdm(imgs)):
#     try:
#         dataset.append(create_mesh(file_path))
#     except:
#         print(file_path) 
        
#     # file_path = "./datasets/svg/032-firewood.svg"
#     # data = create_mesh(file_path)
#     # print(data)
#     # break

In [5]:
imgs = imgs[:2000]
dataset = []
for data in tqdm(multiprocessing.Pool(8).imap_unordered(create_mesh, imgs), total=len(imgs)):
    dataset.append(data)

100%|██████████| 2000/2000 [04:01<00:00,  8.28it/s]


In [6]:
# hyperparameters
torch.manual_seed(16)

batch_size = 32
num_features = 2
num_output = 1
num_epoch = 500

_train = int(len(dataset) * 0.8)
_val = _train + int(len(dataset) * 0.1)
_test = len(dataset) - _val

# create dataloader
train_set, val_set, test_set = dataset[:_train], dataset[_train:_val], dataset[_val:]
train_svg, val_svg, test_svg = imgs[:_train], imgs[_train:_val], imgs[_val:]
train_png, val_png, test_png = png[:_train], png[_train:_val], png[_val:]

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

print(f"Training Data: {len(train_set)}\nValidation Data: {len(val_set)}\nTesting Data: {len(test_set)}")

Training Data: 1600
Validation Data: 200
Testing Data: 200


In [7]:
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')
        self.lin = Linear(in_channels, out_channels, bias=False)
        self.bias = Parameter(torch.Tensor(out_channels))

        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()

    def forward(self, x, edge_index, edge_attr):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-5: Start propagating messages.
        out = self.propagate(edge_index, x=x, norm=norm, edge_attr=edge_attr)

        # Step 6: Apply a final bias vector.
        out += self.bias

        return out

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j
    
class EdgeConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='max') #  "Max" aggregation.
        self.mlp = Seq(Linear(2 * in_channels, out_channels),
                       ReLU(),
                       Linear(out_channels, out_channels))

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        return self.propagate(edge_index, x=x)

    def message(self, x_i, x_j):
        # x_i has shape [E, in_channels]
        # x_j has shape [E, in_channels]

        tmp = torch.cat([x_i, x_j - x_i], dim=1)  # tmp has shape [E, 2 * in_channels]
        return self.mlp(tmp)
    
class MLP(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(in_channels, in_channels*2),
            torch.nn.BatchNorm1d(in_channels*2),
            torch.nn.ReLU(),
            torch.nn.Linear(in_channels*2, out_channels*2),
            torch.nn.BatchNorm1d(out_channels*2),
            torch.nn.ReLU(),
            torch.nn.Linear(out_channels*2, out_channels),
        )

    def forward(self, x):
        return self.layers(x)
    
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(num_features, num_features*2)
        self.conv2 = GCNConv(num_features*2, num_features)
        self.mlp = MLP(num_features, 1)
        
        # self.conv1 = EdgeConv(num_features, 16)
        # self.conv2 = EdgeConv(16, num_output)
        
        

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_attr)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.mlp(x)

        return F.softmax(x)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# criterion = torch.nn.MSELoss(reduction='mean')
criterion = torch.nn.CrossEntropyLoss()

In [11]:
# training
train_losses = []
val_losses = []
best_loss = float('inf')

for epoch in range(2):  # num_epoch
    train_loss = 0
    val_loss = 0
    
    model.train()
    for i, data in enumerate(tqdm(train_loader)):
        data = data.to(device)
        
        optimizer.zero_grad()
        out = model(data)  # out.cpu().detach().numpy().shape = (num_node, 3)
        h = data.h.type(torch.LongTensor).to(device)
        loss = criterion(out, h)
        # loss = criterion(out, data.y)
        
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
    model.eval()
    for i, data in enumerate(tqdm(val_loader)):
        data = data.to(device)
        out = model(data)
        
        h = data.h.type(torch.LongTensor).to(device)
        loss = criterion(out, h)
        # loss = criterion(out, data.y)
        
        val_loss += loss.item()
    
    train_avg = train_loss / len(train_loader)
    val_avg = val_loss / len(val_loader)
    train_losses.append(train_avg)
    val_losses.append(val_avg)
    
    print(f'Epoch {epoch}\tTraining Loss: {train_avg}\tValidation Loss: {val_avg}')
    
    if val_avg < best_loss:
        print(f'Validation Loss Decreased({best_loss:.6f}--->{val_avg:.6f})\tSaving The Model')
        best_loss = val_avg
        torch.save(model.state_dict(), 'best_checkpoint.pth')

  0%|          | 0/50 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
# plot losses
import matplotlib.pyplot as plt

_x = list(range(num_epoch))
plt.plot(_x, train_losses, label='Training Loss')
plt.plot(_x, val_losses, label='Validation Loss')
 
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# testing
model.load_state_dict(torch.load('./best_checkpoint.pth'))
model.eval()

test_loss = 0
for i, data in enumerate(tqdm(test_loader)):
    data = data.to(device)
    out = model(data)
    loss = criterion(out, data.y)
    test_loss += loss.item()
    
print(f'Testing Loss: {test_loss / len(test_loader)}')

In [ ]:
# visualize testing results
vis_loader = DataLoader(test_set, batch_size=1, shuffle=False)

model.load_state_dict(torch.load('./best_checkpoint.pth'))
model.eval()

for data, png in zip(vis_loader, test_png):
    data = data.to(device)
    out = model(data)
    
    pos = data.x.cpu().detach().numpy()[:,3:]
    rgb = out.cpu().detach().numpy()
    
    # get pos and hex color
    n_rgb = rgb * 255
    cc = []
    for r, g, b in n_rgb:
        cc.append("#" + ('{:X}{:X}{:X}').format(int(r), int(g), int(b)))
    
    # # plot output color
    # plt.subplot(121)
    # img = Image.open(png)
    # plt.axis("off")
    # plt.imshow(img)
    
    # plt.subplot(122)
    # # plt.axis([0, 25, 33, -8])
    for (x, y), c in zip(pos, cc):
        plt.scatter(x, y, c=c)
    plt.axis("off")
    plt.show()
    plt.close()
    break